In [1]:
# Runs XGBoost, QDA, GBC
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import xgboost as xgbst

In [24]:
# Read the train and the test data 
# Run the preprocessing notebook to get the csv listed here
train_users = pd.read_csv('train_users_merge_scale.csv')
#test_users = pd.read_csv('test_users.csv')


# Extracting labels from the train data
train_users_labels = train_users.loc[:,'country_destination']
print (train_users_labels.head(n=5))

# Extracting attributes from the train data
train_users_attrs = train_users.iloc[:,:-1]
print(train_users_attrs.head(n=5))

train_users = train_users_attrs
print(train_users.columns)
#train_users = train_users.drop(['id'], axis=1)
train_users = train_users.drop(['Blackberry'], axis=1)
train_users = train_users.drop(['Opera Phone'], axis=1)
labels_df = pd.DataFrame(train_users_labels)

0    0
1    0
2    1
3    2
4    1
Name: country_destination, dtype: int64
   timestamp_first_active    gender       age  signup_method  signup_flow  \
0               -4.380020 -0.927300 -0.163283      -1.596552    -0.427798   
1               -4.357961  1.058047  0.287705      -1.596552    -0.427798   
2               -4.348661 -0.927300  2.317149       0.628333    -0.035009   
3               -4.303076 -0.927300  0.738692      -1.596552    -0.427798   
4               -4.283949 -0.927300  0.625945       0.628333    -0.427798   

   language  affiliate_channel  affiliate_provider  first_affiliate_tracked  \
0 -0.141579          -0.582242           -0.468760                -0.798954   
1 -0.141579           2.556797            0.251719                -0.798954   
2 -0.141579          -0.582242           -0.468760                -0.798954   
3 -0.141579          -0.582242           -0.468760                -0.798954   
4 -0.141579          -0.582242           -0.468760                -

In [8]:
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import make_scorer

def dcg_score(y_true, y_score, k=5):
    
    order = np.argsort(y_score)[::-1]
    y_true = np.take(y_true, order[:k])

    gain = 2 ** y_true - 1

    discounts = np.log2(np.arange(len(y_true)) + 2)
    return np.sum(gain / discounts)


#def ndcg_score(ground_truth, predictions, k=5):
def ndcg_score(te_labels, predict, k):
    
    lb = LabelBinarizer()
    lb.fit(range(12 + 1))
    T = lb.transform(te_labels)

    scores = []

    # Iterate over each y_true and compute the DCG score
    for y_true, y_score in zip(T, predict):
        actual = dcg_score(y_true, y_score, k)
        best = dcg_score(y_true, y_true, k)
        if best == 0:
            best = 0.000000001
        score = float(actual) / float(best)
        scores.append(score)
    return np.mean(scores)


# NDCG Scorer function
ndcg_scorer = make_scorer(ndcg_score, needs_proba=True, k=5)


In [13]:
def folds_to_split(data,targets,train,test):
    data_tr = pd.DataFrame(data).iloc[train]
    data_te = pd.DataFrame(data).iloc[test]
    labels_tr = pd.DataFrame(targets).iloc[train]
    labels_te = pd.DataFrame(targets).iloc[test]
    return [data_tr, data_te, labels_tr, labels_te]

In [29]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn.multiclass import OneVsRestClassifier
from sklearn import linear_model, cross_validation
import xgboost

def ten_fold_xgb(data, labels):
    foldnum = 0
    fold_results_ovr_est = pd.DataFrame()
    fold_results_xgb = pd.DataFrame()
    for train, test in cross_validation.KFold(len(data), n_folds=10, shuffle=True):
        foldnum+=1
        [tr_data, te_data,
         tr_target, te_target] = folds_to_split(data, labels,train,test)
        xgb = xgbst.XGBClassifier(max_depth=5, learning_rate=0.3, n_estimators=25,
                    objective='multi:softprob', subsample=0.5, colsample_bytree=0.7)
        
        #print(tr_target)
        xgb.fit(tr_data.values, tr_target[tr_target.columns.values[0]].values)
        #print(oneVsRest.estimators_)
        
        #perc.fit(heart_tr_data, heart_tr_target[0].values)
        fold_results_xgb.loc[foldnum, 'Accuracy'] = xgb.score(te_data, te_target)
        
        predictions = xgb.predict_proba(te_data)
        score = ndcg_score(te_target.as_matrix(), predictions, 5)
        fold_results_xgb.loc[foldnum, 'ndcg']  = score
        print(score)
            
    
    #Now let's look at the results:
    print ("------xbg accuracy values------")
    print (fold_results_xgb)
    print ("------xgb mean accuracy values------")
    print(fold_results_xgb.mean())
    
ten_fold_xgb(train_users, labels_df)

0.827857150181
0.827922539153
0.831051357283
0.830103425363
0.830900322052
0.829794146731
0.829413793112
0.825797729642
0.829964795489
0.832462151178
------xbg accuracy values------
    Accuracy      ndcg
1   0.645414  0.827857
2   0.643757  0.827923
3   0.648817  0.831051
4   0.647552  0.830103
5   0.646521  0.830900
6   0.645491  0.829794
7   0.645866  0.829414
8   0.637058  0.825798
9   0.645303  0.829965
10  0.650316  0.832462
------xgb mean accuracy values------
Accuracy    0.645610
ndcg        0.829527
dtype: float64


In [52]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn.multiclass import OneVsRestClassifier
from sklearn import linear_model, cross_validation
import xgboost

fold_results_xgb = pd.DataFrame()
test_sets = []
test_set_labels = []
train_sets = []
train_set_labels = []
def ten_fold_xgb_val(data, labels):
    fold_results_ovr_est = pd.DataFrame()
    
    for train, test in cross_validation.KFold(len(data), n_folds=10, shuffle=True):
        [tr_data, te_data,
         tr_target, te_target] = folds_to_split(data, labels,train,test)
        
        train_sets.append(tr_data)
        train_set_labels.append(tr_target)
        test_sets.append(te_data)
        test_set_labels.append(te_target)
    
    depth = [4, 5, 6]
    estimator = [20, 25, 30]
    colsample = [0.5, 0.7]
    foldnum = 0
    for x in range(0, 10):
        foldnum+=1
        [bnb_train, bnb_validation, bnb_train_labels, bnb_validation_labels] = cross_validation.train_test_split(
            train_sets[x], 
            train_set_labels[x], 
            test_size=0.11, 
            random_state=20160121)
        for dep in depth:
            for est in estimator:
                for cs in colsample:
                    xgb = xgbst.XGBClassifier(max_depth=dep, learning_rate=0.3, n_estimators=est,
                                                objective='multi:softprob', subsample=0.5, colsample_bytree=cs)
        
        
                    #print(tr_target)
                    xgb.fit(bnb_train.values, bnb_train_labels[bnb_train_labels.columns.values[0]].values)
                    #print(oneVsRest.estimators_)
        
                    #perc.fit(heart_tr_data, heart_tr_target[0].values)
                    col_name1 = 'Accuracy_' + 'dep' +str(dep) +'_est' +str(est) + '_cs' +str(cs)
                    fold_results_xgb.loc[foldnum, col_name1] = xgb.score(bnb_validation, bnb_validation_labels)
        
                    predictions = xgb.predict_proba(bnb_validation)
                    score = ndcg_score(bnb_validation_labels.as_matrix(), predictions, 5)
                    col_name2 = 'ndcg_' + 'dep' +str(dep) +'_est' +str(est)  + '_cs' +str(cs)
                    fold_results_xgb.loc[foldnum, col_name2]  = score
                    print(str(score) + ': ndcg_' + 'dep' +str(dep) +'_est' +str(est) + '_cs' +str(cs))
            
    
    #Now let's look at the results:
    print ("------xbg accuracy values------")
    print (fold_results_xgb)
    print ("------xgb mean accuracy values------")
    print(fold_results_xgb.mean())
    
ten_fold_xgb_val(train_users, labels_df)

0.83091005915: ndcg_dep4_est20_cs0.5
0.830348312851: ndcg_dep4_est20_cs0.7
0.830454820095: ndcg_dep4_est25_cs0.5
0.829829458372: ndcg_dep4_est25_cs0.7
0.830754727034: ndcg_dep4_est30_cs0.5
0.830313406016: ndcg_dep4_est30_cs0.7
0.830398149083: ndcg_dep5_est20_cs0.5
0.830641409686: ndcg_dep5_est20_cs0.7
0.830538736157: ndcg_dep5_est25_cs0.5
0.830750932297: ndcg_dep5_est25_cs0.7
0.830447993942: ndcg_dep5_est30_cs0.5
0.831277747404: ndcg_dep5_est30_cs0.7
0.830148420412: ndcg_dep6_est20_cs0.5
0.830897545347: ndcg_dep6_est20_cs0.7
0.830742977824: ndcg_dep6_est25_cs0.5
0.830706467337: ndcg_dep6_est25_cs0.7
0.830790206499: ndcg_dep6_est30_cs0.5
0.830440402184: ndcg_dep6_est30_cs0.7
0.829242283501: ndcg_dep4_est20_cs0.5
0.829903461778: ndcg_dep4_est20_cs0.7
0.830095713459: ndcg_dep4_est25_cs0.5
0.829781317948: ndcg_dep4_est25_cs0.7
0.830392484042: ndcg_dep4_est30_cs0.5
0.82988891333: ndcg_dep4_est30_cs0.7
0.829832133665: ndcg_dep5_est20_cs0.5
0.830774939249: ndcg_dep5_est20_cs0.7
0.830437469173

In [53]:
# Setting the attributes which do best in the validation set...

fold_results_xgb2 = pd.DataFrame()
foldnum = 0
for x in range(0, 10):
    
    foldnum+=1
    xgb = xgbst.XGBClassifier(max_depth=5, learning_rate=0.3, n_estimators=25,
                              objective='multi:softprob', subsample=0.5, colsample_bytree=0.7)
    xgb.fit(train_sets[x].values, train_set_labels[x].values.ravel())
    
    fold_results_xgb2.loc[foldnum, 'Accuracy'] = xgb.score(test_sets[x].values,
                                                          test_set_labels[x].values.ravel())
    
    predictions = xgb.predict_proba(test_sets[x].values)
    score = ndcg_score(test_set_labels[x].as_matrix(), predictions, 5)
    fold_results_xgb2.loc[foldnum, 'ndcg']  = score
    
print ("------xbg accuracy and ndcg values------")
print (fold_results_xgb2)
print ("------xgb mean accuracy and ndgc values------")
print(fold_results_xgb2.mean())

------xbg accuracy and ndcg values------
    Accuracy      ndcg
1   0.647381  0.830642
2   0.648630  0.830800
3   0.651253  0.831373
4   0.642773  0.827528
5   0.642164  0.829708
6   0.644647  0.829175
7   0.645069  0.829802
8   0.641602  0.828495
9   0.650223  0.829953
10  0.646381  0.829317
------xgb mean accuracy and ndgc values------
Accuracy    0.646012
ndcg        0.829679
dtype: float64


In [57]:
# QDA performs the worst among all the algorithms

from sklearn.qda import QDA

fold_results_qda = pd.DataFrame()
test_sets = []
test_set_labels = []
train_sets = []
train_set_labels = []
    
def ten_fold_qda(data, labels):
    foldnum = 0
    for train, test in cross_validation.KFold(len(data), n_folds=10, shuffle=True, random_state=20160202):
        [tr_data, te_data,
         tr_target, te_target] = folds_to_split(data, labels,train,test)

        foldnum+=1
        quad_da = QDA()
        #print(tr_target)
        quad_da.fit(tr_data.values, tr_target[tr_target.columns.values[0]].values)
        #print(oneVsRest.estimators_)
        
        fold_results_qda.loc[foldnum, 'Accuracy'] = quad_da.score(te_data, te_target)
        
        predictions = quad_da.predict_proba(te_data)
        score = ndcg_score(te_target.as_matrix(), predictions, 5)
        fold_results_qda.loc[foldnum, 'ndcg']  = score
        #print(score)            
                
    #Now let's look at the results:
    print ("------QDA accuracy and ndcg values------")
    print (fold_results_qda)
    print ("------QDA mean accuracy, ndcg values------")
    print(fold_results_qda.mean())
    
ten_fold_qda(train_users, labels_df)

------QDA accuracy and ndcg values------
    Accuracy      ndcg
1   0.003701  0.021136
2   0.002436  0.020786
3   0.003982  0.026266
4   0.003561  0.020837
5   0.003092  0.020791
6   0.003795  0.023865
7   0.004076  0.023160
8   0.003279  0.021676
9   0.002764  0.019919
10  0.004779  0.023424
------QDA mean accuracy, ndcg values------
Accuracy    0.003546
ndcg        0.022186
dtype: float64


In [36]:
from sklearn.ensemble import GradientBoostingClassifier

def ten_fold_gbc(data, labels):
    foldnum = 0
    fold_results_ovr_est = pd.DataFrame()
    fold_results_gbc = pd.DataFrame()
    for train, test in cross_validation.KFold(len(data), n_folds=10, shuffle=True):
        foldnum+=1
        [tr_data, te_data,
         tr_target, te_target] = folds_to_split(data, labels,train,test)
        
        gbc = GradientBoostingClassifier(n_estimators=100, learning_rate=0.2, max_depth=5, random_state=0)
        
        #print(tr_target)
        gbc.fit(tr_data.values, tr_target[tr_target.columns.values[0]].values)
        #print(oneVsRest.estimators_)
        
        #perc.fit(heart_tr_data, heart_tr_target[0].values)
        fold_results_gbc.loc[foldnum, 'Accuracy'] = gbc.score(te_data, te_target)
        
        predictions = gbc.predict_proba(te_data)
        score = ndcg_score(te_target.as_matrix(), predictions, 5)
        fold_results_gbc.loc[foldnum, 'ndcg']  = score
        print(score)
            
    
    #Now let's look at the results:
    print ("------GradientBoostingClassifier accuracy,ndcg values------")
    print (fold_results_gbc)
    print ("------GradientBoostingClassifier mean accuracy,ndcg values------")
    print(fold_results_gbc.mean())
    
ten_fold_gbc(train_users, labels_df)

0.808405508661
0.823569179662
0.823636935903
0.823817813818
0.825779005229
0.824708311904
0.825328238479
0.821624199873
0.829299174018
0.827329062389
------GradientBoostingClassifier accuracy,ndcg values------
    Accuracy      ndcg
1   0.630329  0.808406
2   0.631717  0.823569
3   0.633825  0.823637
4   0.634809  0.823818
5   0.638651  0.825779
6   0.636449  0.824708
7   0.638135  0.825328
8   0.633076  0.821624
9   0.642539  0.829299
10  0.640993  0.827329
------GradientBoostingClassifier mean accuracy,ndcg values------
Accuracy    0.636052
ndcg        0.823350
dtype: float64
